##### my-zero-to-gbm-proj-assign

In [1]:
import pandas as pd
import numpy as np

In [2]:
inputs = pd.read_csv('inputs.csv',index_col=0)
targets = pd.read_csv('targets.csv',index_col=0)

In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
from sklearn.metrics import r2_score
# def rmspe(y_true, y_pred):
#     return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [5]:
from sklearn.model_selection import ShuffleSplit, RepeatedKFold

In [6]:
# ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=11111)
ss = RepeatedKFold(n_splits=2, n_repeats=2, random_state=11111)

In [7]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
poly_inputs = pd.DataFrame(poly.fit_transform(inputs)).astype('float32')

In [8]:
from sklearn.metrics import mean_squared_error
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [12]:
# # explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting # noqa
# # now you can import normally from model_selection

In [11]:
import sklearn
sklearn.__version__

'0.23.1'

In [13]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
# # explicitly require this experimental feature
# from sklearn.experimental import enable_halving_search_cv # noqa
# # now you can import normally from model_selection
# from sklearn.model_selection import HalvingRandomSearchCV
# from sklearn.model_selection import HalvingGridSearchCV

In [16]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
#    model = LGBMRegressor(random_state=11111,silent=False,first_metric_only= True,metric = "None",feval=[rmspe],force_col_wise=True,mc=[0,1,1,1,1,1,1,1,1,1],**params)
    model = HistGradientBoostingRegressor(random_state=11111,monotonic_cst=[0,1,1,1,1,1,1,1,1,1],verbose=1)
    model.fit(X_train, train_targets)#,early_stopping_rounds=5,eval_set=(X_val,val_targets))
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    train_R2 = round(r2_score(train_targets,pred_train ),3)
    train_RMSPE = round(mean_squared_error(train_targets, pred_train,squared=False),3)
    val_R2 = round(r2_score(val_targets, pred_val),3)
    val_RMSPE = round(mean_squared_error(val_targets, pred_val,squared=False),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

In [22]:
# #modelc = LGBMRegressor(random_state=11111,silent=False,first_metric_only= True,metric = "None",force_col_wise=True,mc=[0,1,1,1,1,1,1,1,1,1])#,bagging_freq= 5, bagging_fraction= 0.75)
modelc = HistGradientBoostingRegressor(random_state=11111,monotonic_cst=[0,1,1,1,1,1,1,1,1,1],verbose=1)
param_grid = {#"loss": ['squared_error','absolute_error'],
              "loss": ['least_squares', 'least_absolute_deviation'],
              "learning_rate" : [0.1,0.01,0.001],
              "max_iter": [100,300],
              "max_leaf_nodes": [16,64,256],
              "min_samples_leaf": [10,20,30],
              "l2_regularization": [0.0,0.001,0.01],
              "max_bins": [63,255],
              "n_iter_no_change": [10,20,50],
              "validation_fraction": [0.1,0.5,0.9],
              "warm_start": [True,False]
            }

In [ ]:
# param_grid = {"boosting_type": ['dart'],
#               "learning_rate" : [0.01,0.05,0.1],
#               "n_estimators": [1000],
#               "min_data_in_leaf": [0,20],
#               "min_split_gain": [0.001],
#               "objective": ["huber"],
#               "num_leaves": [16,64,256,1024],
#               "max_bin": [63,255,1024],
#               "min_sum_hessian_in_leaf": [1.0,100.0],
#               "reg_lambda": [0.01,10.0],
#               "uniform_drop": [True],
#               "monotone_constraints_method": ['advanced'] ,
#               "linear_tree": ['true'],
#               "data_random_seed": [42],
#             }

In [ ]:
# models = []
# model_params = []
# grid_params = []
# for train_idxs, val_idxs in ss.split(poly_inputs):
#    X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
#    X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
#    X_train = np.ascontiguousarray(X_train).reshape(-1,10)
#    train_targets = np.ascontiguousarray(train_targets).ravel()
#    X_val = np.ascontiguousarray(X_val).reshape(-1,10)
#    val_targets = np.ascontiguousarray(val_targets).ravel()
#    grid_search = HalvingGridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,random_state=11111,factor=2,min_resources=1000,error_score='raise').fit(X_train, train_targets)
# #   grid_search = GridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,error_score='raise').fit(X_train, train_targets)
#    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                    train_targets, 
#                                                    X_val, 
#                                                    val_targets, 
#                                                    **grid_search.best_params_)
#    models.append(model)
#    model_params.append(model.get_params())
#    grid_params.append(grid_search.best_params_)
#    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

In [23]:
models = []
model_params = []
grid_params = []
for train_idxs, val_idxs in ss.split(poly_inputs):
   X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
   X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
   X_train = np.ascontiguousarray(X_train).reshape(-1,10)
   train_targets = np.ascontiguousarray(train_targets).ravel()
   X_val = np.ascontiguousarray(X_val).reshape(-1,10)
   val_targets = np.ascontiguousarray(val_targets).ravel()
#   grid_search =  HalvingRandomSearchCV(modelc, param_grid,verbose=1,return_train_score=True,random_state=11111,factor=2,min_resources=1000,error_score='raise').fit(X_train, train_targets)
   grid_search =  RandomizedSearchCV(modelc, param_grid,verbose=1,return_train_score=True,random_state=11111,error_score='raise').fit(X_train, train_targets)
#   grid_search = GridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,error_score='raise').fit(X_train, train_targets)
   model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                   train_targets, 
                                                   X_val, 
                                                   val_targets, 
                                                   **grid_search.best_params_)
   models.append(model)
   model_params.append(model.get_params())
   grid_params.append(grid_search.best_params_)
   print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Binning 0.007 GB of training data: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.172 s
Binning 0.007 GB of validation data: 0.027 s
Fitting gradient boosted rounds:
[1/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00173, val loss: 0.00173, in 0.083s
[2/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00165, val loss: 0.00166, in 0.086s
[3/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00159, val loss: 0.00160, in 0.076s
[4/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00154, val loss: 0.00154, in 0.086s
[5/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00150, val loss: 0.00150, in 0.072s
[6/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00146, val loss: 0.00146, in 0.115s
[7/300] 1 tree, 64 leaves, max depth = 13, train loss: 0.00143, val loss: 0.00143, in 0.154s
[8/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00140, val loss: 0.00141, in 0.113s
[9/300] 1 tree, 64 leaves, max depth = 13, train loss: 0.00138, val loss: 0.00139, in 0.097s
[10/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00137, val loss: 0.

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 17.1min finished


0.238 s
Binning 0.009 GB of validation data: 0.021 s
Fitting gradient boosted rounds:
[1/300] 1 tree, 64 leaves, max depth = 13, train loss: 0.00179, val loss: 0.00178, in 0.089s
[2/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00171, val loss: 0.00170, in 0.107s
[3/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00165, val loss: 0.00164, in 0.103s
[4/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00159, val loss: 0.00158, in 0.074s
[5/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00154, val loss: 0.00154, in 0.081s
[6/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00151, val loss: 0.00150, in 0.078s
[7/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00147, val loss: 0.00147, in 0.070s
[8/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00145, val loss: 0.00144, in 0.098s
[9/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00143, val loss: 0.00142, in 0.087s
[10/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00141, val loss: 0.

...

0.175 s
Binning 0.009 GB of validation data: 0.018 s
Fitting gradient boosted rounds:
[1/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00178, val loss: 0.00179, in 0.072s
[2/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00171, val loss: 0.00171, in 0.067s
[3/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00164, val loss: 0.00164, in 0.069s
[4/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00158, val loss: 0.00159, in 0.069s
[5/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00154, val loss: 0.00154, in 0.131s
[6/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00150, val loss: 0.00150, in 0.108s
[7/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00147, val loss: 0.00147, in 0.067s
[8/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00144, val loss: 0.00144, in 0.129s
[9/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00142, val loss: 0.00142, in 0.112s
[10/300] 1 tree, 64 leaves, max depth = 9, train loss: 0.00140, val loss: 0.0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


1 tree, 64 leaves, max depth = 12, train loss: 0.00173, val loss: 0.00173, in 0.062s
[2/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00165, val loss: 0.00165, in 0.058s
[3/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00159, val loss: 0.00159, in 0.057s
[4/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00154, val loss: 0.00154, in 0.057s
[5/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00149, val loss: 0.00149, in 0.062s
[6/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00146, val loss: 0.00146, in 0.069s
[7/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00143, val loss: 0.00143, in 0.061s
[8/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00140, val loss: 0.00140, in 0.059s
[9/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00138, val loss: 0.00138, in 0.062s
[10/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00137, val loss: 0.00136, in 0.059s
[11/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00135, val loss: 0.

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 14.1min finished


0.229 s
Binning 0.009 GB of validation data: 0.029 s
Fitting gradient boosted rounds:
[1/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00178, val loss: 0.00179, in 0.170s
[2/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00170, val loss: 0.00171, in 0.113s
[3/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00163, val loss: 0.00165, in 0.079s
[4/300] 1 tree, 64 leaves, max depth = 12, train loss: 0.00158, val loss: 0.00159, in 0.070s
[5/300] 1 tree, 64 leaves, max depth = 9, train loss: 0.00153, val loss: 0.00154, in 0.096s
[6/300] 1 tree, 64 leaves, max depth = 9, train loss: 0.00149, val loss: 0.00151, in 0.077s
[7/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00146, val loss: 0.00147, in 0.084s
[8/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00143, val loss: 0.00145, in 0.084s
[9/300] 1 tree, 64 leaves, max depth = 10, train loss: 0.00141, val loss: 0.00143, in 0.084s
[10/300] 1 tree, 64 leaves, max depth = 11, train loss: 0.00140, val loss: 0.00

In [ ]:
model.feature_importances_

In [ ]:
df_gd_par = pd.DataFrame(grid_params)
df_md_par = pd.DataFrame(model_params)

In [ ]:
df_gd_par

In [ ]:
df_md_par